In [2]:
import os
import sys
import numpy as np
import pandas as pd
import datetime
import seaborn as sns
import ogb
import ast

In [15]:
cwd = os.getcwd()
print(cwd)
cwd_parent = os.path.abspath(os.path.join(cwd, os.pardir))
# cwd_parent = os.path.abspath(os.path.join(cwd, '../../'))
print(cwd_parent)

sys.path.append(cwd_parent)

/Users/aliciapliego/Projects/drug_synergy/AI-predict/code
/Users/aliciapliego/Projects/drug_synergy/AI-predict


In [16]:
import deepadr
from deepadr.dataset import *
from deepadr.utilities import *
from deepadr.chemfeatures import *
from deepadr.train_functions_flat import *
from deepadr.model_gnn_ogb import GNN, DeepAdr_SiameseTrf, ExpressionNN

In [3]:
rawdata_dir = '../data/raw/'
processed_dir = '../data/processed/'
up_dir = '..'
targetdata_dir_raw = '/Users/aliciapliego/Projects/drug_synergy/AI-predict/data/processed/dataset_generation/raw'

In [4]:
data = pd.read_csv(os.path.join(targetdata_dir_raw, 'data_pkl.tsv'), sep = '\t')

In [5]:
df_rna = pd.read_csv('../data/processed/gene_expression_data_preprocessed.csv')
df_rna

,Gene_ID,ACH-000277,ACH-000837,ACH-000930,ACH-000615,ACH-000090,ACH-000147,ACH-000849,ACH-000785,ACH-001190,...,ACH-000551,ACH-000720,ACH-001196,ACH-000376,ACH-000267,ACH-000680,ACH-000481,ACH-000856,ACH-000730,ACH-000530
0,1,5.146492,2.981853,4.052242,4.822220,1.263034,4.516646,2.678072,0.847997,3.893362,...,4.624686,0.695994,2.010780,4.305971,4.175525,0.250962,0.704872,4.724105,4.174726,5.621759
1,10,0.124328,0.286881,0.000000,0.000000,0.042644,0.250962,0.163499,0.485427,0.111031,...,0.000000,0.137504,0.464668,0.250962,0.000000,1.807355,0.163499,0.070389,0.056584,0.000000
2,100,0.084064,3.481557,4.149747,2.639232,2.891419,1.726831,3.559492,4.827819,3.538538,...,2.899176,2.017922,5.140370,2.912650,2.147307,3.635754,2.780310,3.970854,3.975447,3.565597
3,1000,0.238787,2.881665,0.622930,3.273516,5.226123,0.056584,1.803227,0.042644,0.910733,...,0.189034,6.029232,0.014355,5.497932,0.356144,0.042644,4.279471,3.085765,0.411426,0.464668
4,10000,0.070389,4.249445,5.250583,4.321207,3.171527,0.111031,3.727920,0.150560,4.725196,...,0.536053,4.532317,3.763412,2.847997,4.167519,0.097611,1.739848,0.855990,3.790772,4.123501
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19174,9991,5.309613,4.902556,4.092546,3.760221,3.324811,4.984134,5.244887,5.058316,4.819668,...,5.868637,4.264536,5.034304,4.020591,5.584662,5.960697,4.760753,4.704872,4.377124,4.574707
19175,9992,0.695994,0.176323,0.613532,0.056584,0.189034,0.575312,0.978196,0.226509,0.000000,...,0.070389,0.263034,0.000000,0.250962,1.014355,0.014355,0.084064,0.286881,0.000000,0.263034
19176,9993,5.157044,5.458448,6.129077,5.028569,3.657640,4.785027,5.043957,4.965323,4.438293,...,5.864929,5.216455,4.590961,5.011675,4.121844,4.756490,4.759688,5.978424,4.508429,5.271276
19177,9994,3.590961,3.317594,2.744161,3.139142,2.260026,3.215679,4.186659,2.480265,3.584963,...,3.671293,3.467279,3.544733,3.157044,4.303781,3.187451,3.196922,4.286142,2.671293,4.100978


In [6]:
df_cnv = pd.read_csv('../data/processed/cnv_data_preprocessed.csv')
df_cnv

,Gene_ID,ACH-000277,ACH-000837,ACH-000930,ACH-000615,ACH-000090,ACH-000147,ACH-000849,ACH-000785,ACH-001190,...,ACH-000551,ACH-000720,ACH-001196,ACH-000376,ACH-000267,ACH-000680,ACH-000481,ACH-000856,ACH-000730,ACH-000530
0,1,1.031555,0.944715,0.756541,1.206992,1.211438,0.768499,1.341251,1.452116,0.905456,...,1.002602,0.925300,1.029293,1.096465,1.008494,0.741872,1.354908,0.980352,0.806532,1.173974
1,10,1.093802,0.546255,0.744336,0.835375,0.784642,0.832593,0.861512,0.640122,0.977407,...,0.982000,0.915472,1.469678,1.037913,0.574689,1.076897,1.076598,1.024159,0.747238,0.707131
2,100,1.787550,1.471560,2.021517,0.925605,1.251417,1.356372,1.037354,1.222511,1.282751,...,1.003322,1.122776,1.075256,1.021755,0.580835,1.416126,1.345187,1.042199,1.296084,0.834743
3,1000,0.685398,0.922892,1.081175,0.817158,1.130757,0.858228,0.818851,0.386752,0.974166,...,1.172393,0.808366,1.165846,1.016473,1.003177,0.810428,1.085301,1.038358,0.950280,1.001396
4,10000,1.108415,1.183689,1.529492,0.994248,1.320970,1.125498,1.235262,1.218143,1.092804,...,1.206254,1.197469,0.985526,1.004714,1.011931,1.047703,1.319866,1.014745,1.046404,1.044751
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19174,9991,1.380481,0.925425,0.767892,0.898100,0.811080,1.381067,0.869019,0.918873,0.973519,...,0.997630,0.964079,1.302591,1.029486,1.013011,1.333506,0.840148,1.017062,1.110135,0.766850
19175,9992,1.042669,0.933492,1.191881,0.951660,1.238675,1.077861,1.309751,0.919539,0.970371,...,1.193927,0.921105,1.009351,1.230122,1.004917,0.791777,0.804547,0.993864,0.856674,0.972384
19176,9993,1.009506,1.258480,1.235767,1.325867,0.751190,0.769720,1.100519,1.088386,1.275583,...,2.066792,0.975385,1.157416,0.996319,1.036152,0.773341,1.055446,1.002974,1.136541,0.772943
19177,9994,1.129907,0.935139,0.779656,1.126231,0.790715,0.869332,1.453172,0.833466,0.977201,...,0.988313,0.886080,0.780440,0.991999,1.031005,0.971278,0.988014,1.020099,0.909038,0.994358


In [7]:
df_mut = pd.read_csv('../data/processed/mutation_data_preprocessed.csv')
df_mut

,Gene_ID,ACH-000277,ACH-000837,ACH-000930,ACH-000615,ACH-000090,ACH-000147,ACH-000849,ACH-000785,ACH-001190,...,ACH-000551,ACH-000720,ACH-001196,ACH-000376,ACH-000267,ACH-000680,ACH-000481,ACH-000856,ACH-000730,ACH-000530
0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,10,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0
2,100,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1000,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,...,2.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0
4,10000,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,...,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19174,9991,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
19175,9992,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
19176,9993,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
19177,9994,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [8]:
rna_array = np.array([df_rna[str(c)].values for c in data["Cell_ID"]])
rna_array.shape

(233326, 19179)

In [9]:
cnv_array = np.array([df_cnv[str(c)].values for c in data["Cell_ID"]])
cnv_array.shape

(233326, 19179)

In [10]:
mut_array = np.array([df_mut[str(c)].values for c in data["Cell_ID"]])
mut_array.shape

(233326, 19179)

In [11]:
omics_array = np.array([np.array(df_rna),np.array(df_cnv), np.array(df_mut)])

In [12]:
omics_array.shape

(3, 19179, 120)

In [17]:
ReaderWriter.dump_data(omics_array, os.path.join(targetdata_dir_raw, 'omics.pkl'))

In [34]:
targetdata_dir = create_directory(os.path.join(processed_dir, 'dataset_generation'))


path_current_dir /Users/aliciapliego/Projects/drug_synergy/AI-predict/deepadr


In [35]:
%%time

# In order to generate a new Dataset, first remove the generated files above

dataset = MoleculeDataset(root=targetdata_dir, dataset='tdcSynergy')

CPU times: user 4.62 ms, sys: 643 ms, total: 647 ms
Wall time: 654 ms


### Explore the data generated by the MoleculeDataset function

In [23]:
geom_filter = torch.load('/Users/aliciapliego/Projects/drug_synergy/AI-predict/data/processed/dataset_generation/processed/geometric_data_processed.pt') 


In [24]:
geom_filter

(PairData(edge_index_a=[2, 15305430], x_a=[7069479, 9], edge_attr_a=[15305430, 3], edge_index_b=[2, 13680958], x_b=[6405928, 9], edge_attr_b=[13680958, 3], id=[233326], y=[233326, 4]),
 defaultdict(dict,
             {'edge_index_a': tensor([       0,       66,      132,  ..., 15305230, 15305300, 15305430]),
              'x_a': tensor([      0,      30,      60,  ..., 7069391, 7069422, 7069479]),
              'edge_attr_a': tensor([       0,       66,      132,  ..., 15305230, 15305300, 15305430]),
              'edge_index_b': tensor([       0,       72,      134,  ..., 13680754, 13680856, 13680958]),
              'x_b': tensor([      0,      31,      62,  ..., 6405836, 6405882, 6405928]),
              'edge_attr_b': tensor([       0,       72,      134,  ..., 13680754, 13680856, 13680958]),
              'id': tensor([     0,      1,      2,  ..., 233324, 233325, 233326]),
              'y': tensor([     0,      1,      2,  ..., 233324, 233325, 233326])}))

In [26]:
pre_transform = torch.load('/Users/aliciapliego/Projects/drug_synergy/AI-predict/data/processed/dataset_generation/processed/pre_transform.pt') 
pre_transform

'None'

In [28]:
pre_filter = torch.load('/Users/aliciapliego/Projects/drug_synergy/AI-predict/data/processed/dataset_generation/processed/pre_filter.pt') 
pre_filter

'None'

## Hyperparmeters

In [36]:
print()
print(f'Dataset: {dataset}:')
print('====================')
print(f'Number of graphs: {len(dataset)}')
print(f'Number of features: {dataset.num_features}')
print(f'Number of classes: {dataset.num_classes}')


Dataset: MoleculeDataset(233326):
Number of graphs: 233326
Number of features: 9
Number of classes: 4


In [37]:
# used_dataset = dataset

# If you want to use a smaller subset of the dataset for testing
smaller_dataset_len = int(len(dataset)/1)
used_dataset = dataset[:smaller_dataset_len]

In [40]:
used_dataset

MoleculeDataset(233326)

In [45]:
device_cpu = get_device(to_gpu=False)
n_gpu = torch.cuda.device_count()
n_gpu

0

In [47]:
fold_partitions = get_stratified_partitions(used_dataset.data.y[:smaller_dataset_len],
                                            num_folds=5, valid_set_portion=0.1, random_state=42)

ValueError: Supported target types are: ('binary', 'multiclass'). Got 'multiclass-multioutput' instead.

In [ ]:
def run_exp(queue, used_dataset, gpu_num, tp, exp_dir, partition): #
    
    num_classes = 2
    
    targetdata_dir_raw = os.path.abspath(exp_dir + "/../../raw")
    targetdata_dir_processed = os.path.abspath(exp_dir + "/../../processed")
    
    state_dict_dir = os.path.join(exp_dir, 'modelstates')
    
    #device_gpu = get_device(True, index=gpu_num)
    #print("gpu:", device_gpu)
    
    # Serialize data into file:
    json.dump( tp, open( exp_dir + "/hyperparameters.json", 'w' ) )
    
    tp['nonlin_func'] = nn.ReLU()
    
    expression_scaler = TorchStandardScaler()
    expression_scaler.fit(used_dataset.data.expression[partition['train']])
    
    train_dataset = Subset(used_dataset, partition['train'])
    val_dataset = Subset(used_dataset, partition['validation'])
    test_dataset = Subset(used_dataset, partition['test'])
    
    train_loader = DataLoader(train_dataset, batch_size=tp["batch_size"], shuffle=True, follow_batch=['x_a', 'x_b'])
    valid_loader = DataLoader(val_dataset, batch_size=tp["batch_size"], shuffle=False, follow_batch=['x_a', 'x_b'])
    test_loader = DataLoader(test_dataset, batch_size=tp["batch_size"], shuffle=False, follow_batch=['x_a', 'x_b'])
    
    loaders = {"train": train_loader, "valid": valid_loader, "test": test_loader}

    gnn_model = GNN(gnn_type = tp["gnn_type"], 
                num_layer = tp["num_layer"], 
                emb_dim = tp["emb_dim"], 
                drop_ratio = 0.5, 
                JK = "multilayer", #last
                graph_pooling = tp["graph_pooling"],
                virtual_node = False,
                with_edge_attr=False).to(device=device_gpu, dtype=fdtype)


    expression_model = DeepSynergy(D_in=(2*tp["emb_dim"])+tp["expression_input_size"],
                                   H1=tp['exp_H1'], H2=tp['exp_H2'], drop=tp['p_dropout']).to(device=device_gpu, dtype=fdtype)

    gene_attn_model = GeneEmbAttention(input_dim=tp["expression_input_size"]).to(device=device_gpu, dtype=fdtype)

    models_param = list(gnn_model.parameters()) + list(expression_model.parameters()) + list(gene_attn_model.parameters())


    model_name = "ogb"
    models = [(gnn_model, f'{model_name}_GNN'),
              (expression_model, f'{model_name}_Expression'),

              (gene_attn_model, f'{model_name}_GeneAttn'),
             ]
    

    y_weights = compute_class_weights(used_dataset.data.y[partition['train']])
    class_weights = torch.tensor(y_weights).type(fdtype).to(device_gpu)

    # from IPython.display import Javascript
    # display(Javascript('''google.colab.output.setIframeHeight(0, true, {maxHeight: 300})'''))

    num_iter = len(train_loader)  # num_train_samples/batch_size
    c_step_size = int(np.ceil(5*num_iter))  # this should be 2-10 times num_iter

    base_lr = tp['base_lr']
    max_lr = tp['max_lr_mul']*base_lr  # 3-5 times base_lr
    optimizer = torch.optim.Adam(models_param, weight_decay=tp["l2_reg"], lr=base_lr)
    cyc_scheduler = torch.optim.lr_scheduler.CyclicLR(optimizer, base_lr, max_lr, step_size_up=c_step_size,
                                                    mode='triangular', cycle_momentum=False)

    loss_nlll = torch.nn.NLLLoss(weight=class_weights, reduction='mean')  # negative log likelihood loss
    loss_contrastive = ContrastiveLoss(0.5, reduction='mean')
    
    
    valid_curve_aupr = []
    test_curve_aupr = []
    train_curve_aupr = []
    
    valid_curve_auc = []
    test_curve_auc = []
    train_curve_auc = []
    
    best_fscore = 0
    best_epoch = 0
      

    for epoch in range(tp["num_epochs"]):
        print("=====Epoch {}".format(epoch))
        print('Training...')
        
        for m, m_name in models:
            m.train()

        for i_batch, batch in enumerate(tqdm(train_loader, desc="Iteration")):
            batch = batch.to(device_gpu)

            h_a = gnn_model(batch.x_a, batch.edge_index_a, batch.edge_attr_a, batch.x_a_batch)
            h_b = gnn_model(batch.x_b, batch.edge_index_b, batch.edge_attr_b, batch.x_b_batch)
            
            expression_norm = expression_scaler.transform_ondevice(batch.expression, device=device_gpu) 
            h_e, _ = gene_attn_model(expression_norm.type(fdtype))
            
            triplet = torch.cat([h_a, h_b, h_e], axis=-1)

            logsoftmax_scores = expression_model(triplet)

            loss = loss_nlll(logsoftmax_scores, batch.y.type(torch.long))            

            loss.backward()  # Derive gradients.
            optimizer.step()  # Update parameters based on gradients.
            cyc_scheduler.step() # after each batch step the scheduler
            optimizer.zero_grad()  # Clear gradients.

        print('Evaluating...')

        perfs = {}

        for dsettype in ["train", "valid"]:
            for m, m_name in models:
                m.eval()

            pred_class = []
            ref_class = []
            prob_scores = []
            
            l_ids = []
           


        #     for data in loader:  # Iterate in batches over the training/test dataset.
            for i_batch, batch in enumerate(tqdm(loaders[dsettype], desc="Iteration")):
                batch = batch.to(device_gpu)
                h_a = gnn_model(batch.x_a, batch.edge_index_a, batch.edge_attr_a, batch.x_a_batch)
                h_b = gnn_model(batch.x_b, batch.edge_index_b, batch.edge_attr_b, batch.x_b_batch)

                expression_norm = expression_scaler.transform_ondevice(batch.expression, device=device_gpu) 
                h_e, _ = gene_attn_model(expression_norm.type(fdtype))


                triplet = torch.cat([h_a, h_b, h_e], axis=-1)
                
                logsoftmax_scores = expression_model(triplet)


                __, y_pred_clss = torch.max(logsoftmax_scores, -1)

                y_pred_prob  = torch.exp(logsoftmax_scores.detach().cpu()).numpy()

                pred_class.extend(y_pred_clss.view(-1).tolist())
                ref_class.extend(batch.y.view(-1).tolist())
                prob_scores.append(y_pred_prob)
                l_ids.extend(batch.id.view(-1).tolist())

            prob_scores_arr = np.concatenate(prob_scores, axis=0)

            dset_perf = perfmetric_report(pred_class, ref_class, prob_scores_arr[:,1], epoch,
                                          outlog = os.path.join(exp_dir, dsettype + ".log"))
            
            perfs[dsettype] = dset_perf
            
            if (dsettype=="valid"):
                
                fscore = F_score(perfs['valid'].s_aupr, perfs['valid'].s_auc)
                if (fscore > best_fscore):
                    best_fscore = fscore
                    best_epoch = epoch
                    
                    for m, m_name in models:
                        torch.save(m.state_dict(), os.path.join(state_dict_dir, '{}.pkl'.format(m_name)))

        print({'Train': perfs['train'], 'Validation': perfs['valid']})

        
        train_curve_aupr.append(perfs['train'].s_aupr)
        valid_curve_aupr.append(perfs['valid'].s_aupr)
        test_curve_aupr.append(0.0)

        
        train_curve_auc.append(perfs['train'].s_auc)
        valid_curve_auc.append(perfs['valid'].s_auc)
        test_curve_auc.append(0.0)


    print('Finished training and validating!')
        
        
    for dsettype in ["test"]:
        
        if(len(os.listdir(state_dict_dir)) > 0):  # load state dictionary of saved models
            for m, m_name in models:
                m.load_state_dict(torch.load(os.path.join(state_dict_dir, '{}.pkl'.format(m_name)), map_location=device_gpu))

        
        for m, m_name in models:
            m.eval()

        pred_class = []
        ref_class = []
        prob_scores = []

        l_ids = []


    #     for data in loader:  # Iterate in batches over the training/test dataset.
        for i_batch, batch in enumerate(tqdm(loaders[dsettype], desc="Iteration")):
            batch = batch.to(device_gpu)
            h_a = gnn_model(batch.x_a, batch.edge_index_a, batch.edge_attr_a, batch.x_a_batch)
            h_b = gnn_model(batch.x_b, batch.edge_index_b, batch.edge_attr_b, batch.x_b_batch)

            expression_norm = expression_scaler.transform_ondevice(batch.expression, device=device_gpu) 
            h_e, _ = gene_attn_model(expression_norm.type(fdtype))


            triplet = torch.cat([h_a, h_b, h_e], axis=-1)

            logsoftmax_scores = expression_model(triplet)


            __, y_pred_clss = torch.max(logsoftmax_scores, -1)

            y_pred_prob  = torch.exp(logsoftmax_scores.detach().cpu()).numpy()

            pred_class.extend(y_pred_clss.view(-1).tolist())
            ref_class.extend(batch.y.view(-1).tolist())
            prob_scores.append(y_pred_prob)
            l_ids.extend(batch.id.view(-1).tolist())

        prob_scores_arr = np.concatenate(prob_scores, axis=0)

        dset_perf = perfmetric_report(pred_class, ref_class, prob_scores_arr[:,1], epoch,
                                      outlog = os.path.join(exp_dir, dsettype + ".log"))

        perfs[dsettype] = dset_perf

        if (dsettype=="test"):

            predictions_df = build_predictions_df(l_ids, ref_class, pred_class, prob_scores_arr)
            predictions_df.to_csv(os.path.join(exp_dir, 'predictions', f'epoch_{epoch}_predictions_{dsettype}.csv'))
            
        print({'Test': perfs['test']})

        test_curve_aupr.pop()
        test_curve_aupr.append(perfs['test'].s_aupr)

        test_curve_auc.pop()
        test_curve_auc.append(perfs['test'].s_auc)

    print('Finished testing!')

    df_curves = pd.DataFrame(np.array([train_curve_aupr, valid_curve_aupr, test_curve_aupr,
                                       train_curve_auc, valid_curve_auc, test_curve_auc]).T)
    df_curves.columns = ['train_aupr', 'valid_aupr', 'test_aupr', 'train_auc', 'valid_auc', 'test_auc']
    df_curves.index.name = "epoch"
    df_curves.to_csv(exp_dir + "/curves.csv")
    sns.lineplot(data=df_curves).figure.savefig(exp_dir + "/curves.png")
    
    queue.put(gpu_num)